# Final Project
#### Assigned: 2019-05-09
#### Due EoD: 2019-05-22

Intraday ETF trading based on 10-sec aggregated equity market data.

#### Name:  Mengheng Xue
#### NetID: mx586

## Data Set

SPY is an Exchange Traded Fund (ETF) that replicates the S&P 500 index, and trades in exchanges like ordinary equity.  
SPY is the most liquid (heavily traded) equity asset in the US.  

For all trading days in June 2018 we have aggregated in 10-sec intervals the trading activity in SPY across all exchanges.  
The dataset is provided in the file `Resources/Data/spy-10sec-201806.csv` at the class site.   
Each row corresponds to a specific 10-sec interval for the corresponding trading day.  
A row reports trading activity within the time period that *ends* at the corresponding interval.  

Regular trading hours in the US are from 09:30 to 16:00.  
You will notice that the 09:30 and 16:00 intervals have much larger traded volume relative to neighboring ones.  
This is because the market opens at 09:30 with an opening auction and closes at 16:00 with a closing auction.  
The consolidated limit order book is the collection of all quotes to buy (bid) and to sell (ask).  
Bid and ask prices/sizes are quotes, i.e. proposals to trade a given quantity at a given price.  
VWAP is the volume weighted average of prices at which trades actually occurred.  




|Column              | Description  |
---------------------|--------------|
|volume              | Number of shares traded within the interval  ($\sum_i V_i $,   summing over all trades $i$)          |
|vwap                | Volume Weighted Average Price ( $VWAP = \sum_i V_i P_i / \sum_i V_i$,   summing over all trades $i$) |
|lowPx/highPx        | Lowest and highest trade prices within the interval                               |
|lastBidPx/lastAskPx | Last bid and ask price in the interval                         |
|lastBidSz/lastAskSz | Last bid and ask sizes (in hundreds of shares) in the interval |


#### Feature engineering

Derived quantities of interest are

1. The Close Location Value (CLV) indicator. This is an intraday adaptation of a classic technical indicator. It is defined as  
$$
CLV_t = \frac{VWAP_t - (lowPx_t + highPx_t)/2}{(highPx_t - lowPx_t)/2}
$$
It measures the location of the VWAP within interval $t$, relative to the mid-point price between low and high price.  

2. The last *quote imbalance* of interval $t$, defined as 
$$
Imbal_t = \frac{lastBidSz_t - lastAskSz_t}{lastBidSz_t + lastAskSz_t}
$$
By construction $-1 \le Imbal \le 1$.  
When $Imbal \rightarrow 1$, there is much more interest to buy than to sell. 
Conversely, when $Imbal \rightarrow -1$ there is much more interest to sell than to buy.  

3. The log-transformed volume defined as $logVolume = log10(Volume)$  
When working with volume-like quantities (non-negative) taking logs is a common normalization.  
Either base 10 or natural logs can be used, base 10 logs may be easier to interpret.

We are also interested in the $N$-period forward return in basis points
$$
\mathrm{fwdRetNBps}_t = 10000 * \left(\frac{VWAP_{t+N}}{VWAP_t} - 1\right)
$$

## Problem (100 points)

#### Objective
+ Construct ML models that use features derived from market observables, to predict price direction in future periods
+ Assess the models using 10-sec as well as 1-min aggregation periods

#### Data Preparation (30 points)
+ Load the SPY 10-sec data
+ Calculate the CLV and the last quote imbalance for each interval. If highPx is equal to lowPx for an interval, set the CLV value to 0.
+ Calculate the 1-period forward VWAP returns in basis points
+ Split the dataset into training sample with the first 16 days, and testing sample with the remaining 4 trading days.
+ Use the average VWAP in the training set a reference price. Call this $\mathrm{RefPx}$.
+ Compute a cutoff return in basis points as:  $\mathrm{cutRetBps} = 10000 * (0.02) / \mathrm{RefPx}$  
  This return corresponds to VWAP movement of twice the typical bid-ask spread (i.e 2 * 0.01)
+ Add a new column called pxDir1 (price direction) and label the 1-period forward price movement as follows:  
  * If fwdRet1Bps > cutRetBps then pxDir1 = +1  
  * If abs(fwdRet1Bps) <= cutRetBps then pxDir1 = 0  
  * If fwdRet1Bps < - cutRet1Bps then pxDir1 = -1  
  Therefore pxDir is a class variable taking values in the set $\{-1, 0, 1\}$.
+ Re-aggregate the 10-sec data into 1-min data and store them in a new data frame.
+ Repeat the process above (CLV, quote imbalance, forward returns, price direction labeling) with the 1-min data.

#### Modeling (60 points)

+ Exploratory data analysis (EDA) which should contain:   
    * univariate distributions of features: logVolume, clv, imbalance  
    * univariate distributions of targets: fwdRet1Bps, pxDir1  
    * any other distribution that may reveal a relationship between target and features  
    * correlation heat map  
+ Construction of a *baseline* model, to be used as a reference.  
  The baseline model predicts the price direction class $C=\{-1, 0, 1\}$ randomly using the class empirical probability of occurence.  
$$
\mathbb{P}(C=\pm 1) = \frac{N_{train}(C=\pm 1)}{N_{train}}, \quad 
\mathbb{P}(C=0) = \frac{N_{train}(C=0)}{N_{train}}
$$
Estimate the empirical probabilities of the baseline model using the training set.  
Make predictions for pxDir1 (simply sample the multinomial distribution) and use the testing set to report  

|Model               | Accuracy  | Precision  | Recall | F1wght | F1micro | 
---------------------|-----------|------------|--------|---------|---------|
|Baseline            | ...       | ...        | ...    | ...     | ...     |

Precision, Recall and F1wght should be measured "weighted" to account for class occurence and potential imbalance.  
F1micro is the "micro" F1 score, i.e. it first computes total true/false positives/negatives first and then computes the F1 score.

+ Construct *two* models, of which one should be neural net based.  
  The other could be any of the classic ML models (Logistic, SVM, Forest, AdaBoost, ...)  
  Train and tune the models in order to forecast the target variable pxDir1.  
  Evaluate the models on the test sample and add their performance metrics to the table above.  
  
+ Reaggregate the data using 1-min intervals and repeat the model runs 

+ Present your conclusions about the best model on the 10-sec and 1-min aggregated data

#### Extra Improvement (10 points)

Attempt to improve model performance by introducing one extra feature variable, derived from the existing market data.  
The extra variable could be either some kind of moving average or an intraday adaptation of a technical indicator.  
Measure the performance improvement for the 10-sec and 1-min dataset.

#### Submission Format

* Follow the submission rules for homeworks.  
* The main document is this notebook, describing the methodology and the conclusions.  
* Make sure your notebook runs using the standard packages we used in class: numpy/scipy/sklearn/statsmodels/tensorflow/keras.
* You can write your own utility classes and functions in separate source code files and import them into this notebook.  
* Assume that your separate source code files will be side-by-side with the notebook. 
* If you are submitting multiple files, put them in a zip archive.

Good luck.

## Solution

### Import Libraries 

In [1]:
import numpy as np
import pandas as pd

### Data Preparation 

In [2]:
names = ['date', 'time', 'sym', 'volume', 'vwap', 'lowPx', 'highPx', 'lastBidPx', 'lastAskPx', 'lastBidSz', 'lastAskSz']
dataset = pd.read_csv('spy-10sec-201806.csv', skiprows=1, names=names)
df_10sec = dataset.copy()
df_10sec.head(10)

,date,time,sym,volume,vwap,lowPx,highPx,lastBidPx,lastAskPx,lastBidSz,lastAskSz
0,2018-06-01,09:30:00,SPY,395424,272.459140,272.320007,272.489990,272.359985,272.380005,14,100
1,2018-06-01,09:30:10,SPY,55692,272.395593,272.339996,272.489990,272.399994,272.410004,9,50
2,2018-06-01,09:30:20,SPY,85164,272.443104,272.390015,272.470001,272.450012,272.459991,18,7
3,2018-06-01,09:30:30,SPY,26973,272.441112,272.420013,272.459991,272.429993,272.440002,5,20
4,2018-06-01,09:30:40,SPY,77809,272.440219,272.410004,272.480011,272.420013,272.429993,1,13
5,2018-06-01,09:30:50,SPY,20625,272.439861,272.429993,272.454987,272.440002,272.459991,100,37
6,2018-06-01,09:31:00,SPY,59681,272.446105,272.390015,272.480011,272.410004,272.420013,21,50
7,2018-06-01,09:31:10,SPY,22027,272.432679,272.399994,272.456696,272.420013,272.429993,8,30
8,2018-06-01,09:31:20,SPY,49068,272.442694,272.420013,272.470001,272.429993,272.440002,4,130
9,2018-06-01,09:31:30,SPY,58974,272.502645,272.440002,272.549988,272.540009,272.549988,16,4


In [3]:
# funciton to calculate Close Location Value
def calculate_clt(vwap, lowPx, highPx):
    if lowPx == highPx:
        clt = 0
    else:
         clt = (vwap-(lowPx+highPx)/2) / ((highPx-lowPx)/2)
    return clt

# function to calculate last quote imbalance
def calculate_Imbal(lastBidSz, lastAskSz):
    imbal = (lastBidSz - lastAskSz)/(lastBidSz+lastAskSz)
    return imbal

# function to calculate N-period forward return in basis points, default is 1-period 
def calculate_fwdRetNBps(df, period=1):
    for i in range(0, len(df)-period):
        df.loc[i, 'fwdRet1Bps'] = 10000 * (df.loc[i+period, 'vwap']/df.loc[i, 'vwap'] - 1)
    return df 

df_10sec['clt'] = df_10sec.apply(lambda x: calculate_clt(x.vwap, x.lowPx, x.highPx), axis=1)
df_10sec['Imbal'] = df_10sec.apply(lambda x: calculate_Imbal(x.lastBidSz, x.lastAskSz), axis=1)
df_10sec = calculate_fwdRetNBps(df_10sec)
df_10sec.head(10)

,date,time,sym,volume,vwap,lowPx,highPx,lastBidPx,lastAskPx,lastBidSz,lastAskSz,clt,Imbal,fwdRet1Bps
0,2018-06-01,09:30:00,SPY,395424,272.459140,272.320007,272.489990,272.359985,272.380005,14,100,0.637020,-0.754386,-2.332345
1,2018-06-01,09:30:10,SPY,55692,272.395593,272.339996,272.489990,272.399994,272.410004,9,50,-0.258679,-0.694915,1.744181
2,2018-06-01,09:30:20,SPY,85164,272.443104,272.390015,272.470001,272.450012,272.459991,18,7,0.327454,0.440000,-0.073097
3,2018-06-01,09:30:30,SPY,26973,272.441112,272.420013,272.459991,272.429993,272.440002,5,20,0.055529,-0.600000,-0.032791
4,2018-06-01,09:30:40,SPY,77809,272.440219,272.410004,272.480011,272.420013,272.429993,1,13,-0.136794,-0.857143,-0.013157
5,2018-06-01,09:30:50,SPY,20625,272.439861,272.429993,272.454987,272.440002,272.459991,100,37,-0.210373,0.459854,0.229212
6,2018-06-01,09:31:00,SPY,59681,272.446105,272.390015,272.480011,272.410004,272.420013,21,50,0.246509,-0.408451,-0.492796
7,2018-06-01,09:31:10,SPY,22027,272.432679,272.399994,272.456696,272.420013,272.429993,8,30,0.152888,-0.578947,0.367606
8,2018-06-01,09:31:20,SPY,49068,272.442694,272.420013,272.470001,272.429993,272.440002,4,130,-0.092555,-0.940299,2.200507
9,2018-06-01,09:31:30,SPY,58974,272.502645,272.440002,272.549988,272.540009,272.549988,16,4,0.139112,0.600000,0.852283


In [18]:
# function calculate price direction values {-1, 0, 1}
def calculate_pxDir1(fwdRet1Bps, cutRetBps):
    if fwdRet1Bps > cutRetBps:
        pxDir1 = 1
    elif abs(fwdRet1Bps) <= cutRetBps:
        pxDir1 = 0
    else:
        pxDir1 = -1
    return pxDir1

# function to separate dataset into 16 days training set and 4 days test set
def train_test_split(df):
    train_mask = (df['date'] <= '2018-06-22')
    test_mask = (df['date'] >= '2018-06-25')
    train_set = df.loc[train_mask]
    test_set = df.loc[test_mask]
    test_set = test_set.reset_index(inplace=True, drop=True)
    
    RefPx = train_set['vwap'].mean()  # reference price
    cutRetBps = 10000*0.02/RefPx # cutoff return 
    df['pxDir1'] = df.apply(lambda x: calculate_pxDir1(x.fwdRet1Bps, cutRetBps), axis=1)
    return df, train_set, test_set

df_10sec, train_set_10sec, test_set_10sec = train_test_split(df_10sec)
df_10sec.head(10)

,date,time,sym,volume,vwap,lowPx,highPx,lastBidPx,lastAskPx,lastBidSz,lastAskSz,clt,Imbal,fwdRet1Bps,pxDir1
0,2018-06-01,09:30:00,SPY,395424,272.459140,272.320007,272.489990,272.359985,272.380005,14,100,0.637020,-0.754386,-2.332345,-1
1,2018-06-01,09:30:10,SPY,55692,272.395593,272.339996,272.489990,272.399994,272.410004,9,50,-0.258679,-0.694915,1.744181,1
2,2018-06-01,09:30:20,SPY,85164,272.443104,272.390015,272.470001,272.450012,272.459991,18,7,0.327454,0.440000,-0.073097,0
3,2018-06-01,09:30:30,SPY,26973,272.441112,272.420013,272.459991,272.429993,272.440002,5,20,0.055529,-0.600000,-0.032791,0
4,2018-06-01,09:30:40,SPY,77809,272.440219,272.410004,272.480011,272.420013,272.429993,1,13,-0.136794,-0.857143,-0.013157,0
5,2018-06-01,09:30:50,SPY,20625,272.439861,272.429993,272.454987,272.440002,272.459991,100,37,-0.210373,0.459854,0.229212,0
6,2018-06-01,09:31:00,SPY,59681,272.446105,272.390015,272.480011,272.410004,272.420013,21,50,0.246509,-0.408451,-0.492796,0
7,2018-06-01,09:31:10,SPY,22027,272.432679,272.399994,272.456696,272.420013,272.429993,8,30,0.152888,-0.578947,0.367606,0
8,2018-06-01,09:31:20,SPY,49068,272.442694,272.420013,272.470001,272.429993,272.440002,4,130,-0.092555,-0.940299,2.200507,1
9,2018-06-01,09:31:30,SPY,58974,272.502645,272.440002,272.549988,272.540009,272.549988,16,4,0.139112,0.600000,0.852283,1


In [38]:
df_1min = dataset.copy()
df_1min['dateTime'] =  pd.to_datetime(df_1min['date' ] + ' ' + df_1min['time'])
df_1min.set_index('dateTime', inplace=True)
df_1min = df_1min.resample('1T').sum() 
df_1min = df_1min.reset_index()
df_1min['date'] = df_1min['dateTime'].dt.strftime('%d/%m/%Y')
df_1min['time'] = df_1min['dateTime'].dt.strftime('%H:%M:%S')
df_1min.head(10)

,dateTime,volume,vwap,lowPx,highPx,lastBidPx,lastAskPx,lastBidSz,lastAskSz,date,time
0,2018-06-01 09:30:00,661687,1634.619029,1634.310028,1634.844971,1634.500000,1634.579987,147,227,01/06/2018,09:30:00
1,2018-06-01 09:31:00,360086,1634.849434,1634.650024,1635.056702,1634.850006,1634.910004,53,356,01/06/2018,09:31:00
2,2018-06-01 09:32:00,270282,1635.050663,1634.890015,1635.257690,1634.980011,1635.059998,71,256,01/06/2018,09:32:00
3,2018-06-01 09:33:00,311647,1634.898663,1634.751465,1635.056732,1634.830017,1634.899994,231,264,01/06/2018,09:33:00
4,2018-06-01 09:34:00,222903,1635.347277,1635.149994,1635.474976,1635.339966,1635.399994,241,182,01/06/2018,09:34:00
5,2018-06-01 09:35:00,352337,1636.444594,1636.097198,1636.639984,1636.519989,1636.590027,157,454,01/06/2018,09:35:00
6,2018-06-01 09:36:00,285776,1636.457849,1636.233124,1636.609985,1636.360016,1636.420013,314,199,01/06/2018,09:36:00
7,2018-06-01 09:37:00,246438,1636.203379,1636.020020,1636.360016,1636.140015,1636.199982,33,339,01/06/2018,09:37:00
8,2018-06-01 09:38:00,130013,1636.066704,1635.959045,1636.200012,1636.060028,1636.130005,212,393,01/06/2018,09:38:00
9,2018-06-01 09:39:00,167173,1636.199338,1636.065521,1636.335052,1636.190002,1636.270020,283,282,01/06/2018,09:39:00


In [20]:
df_1min['clt'] = df_1min.apply(lambda x: calculate_clt(x.vwap, x.lowPx, x.highPx), axis=1)
df_1min['Imbal'] = df_1min.apply(lambda x: calculate_Imbal(x.lastBidSz, x.lastAskSz), axis=1)
df_1min = calculate_fwdRetNBps(df_1min)
df_1min, train_set_1min, test_set_1min = train_test_split(df_1min)
df_1min.head(10)

,date,time,sym,volume,vwap,lowPx,highPx,lastBidPx,lastAskPx,lastBidSz,lastAskSz,clt,Imbal,fwdRet1Bps,pxDir1
0,2018-06-01,09:30:00,SPY,395424,272.459140,272.320007,272.489990,272.359985,272.380005,14,100,0.637020,-0.754386,-2.332345,-1
1,2018-06-01,09:30:10,SPY,55692,272.395593,272.339996,272.489990,272.399994,272.410004,9,50,-0.258679,-0.694915,1.744181,1
2,2018-06-01,09:30:20,SPY,85164,272.443104,272.390015,272.470001,272.450012,272.459991,18,7,0.327454,0.440000,-0.073097,0
3,2018-06-01,09:30:30,SPY,26973,272.441112,272.420013,272.459991,272.429993,272.440002,5,20,0.055529,-0.600000,-0.032791,0
4,2018-06-01,09:30:40,SPY,77809,272.440219,272.410004,272.480011,272.420013,272.429993,1,13,-0.136794,-0.857143,-0.013157,0
5,2018-06-01,09:30:50,SPY,20625,272.439861,272.429993,272.454987,272.440002,272.459991,100,37,-0.210373,0.459854,0.229212,0
6,2018-06-01,09:31:00,SPY,59681,272.446105,272.390015,272.480011,272.410004,272.420013,21,50,0.246509,-0.408451,-0.492796,0
7,2018-06-01,09:31:10,SPY,22027,272.432679,272.399994,272.456696,272.420013,272.429993,8,30,0.152888,-0.578947,0.367606,0
8,2018-06-01,09:31:20,SPY,49068,272.442694,272.420013,272.470001,272.429993,272.440002,4,130,-0.092555,-0.940299,2.200507,1
9,2018-06-01,09:31:30,SPY,58974,272.502645,272.440002,272.549988,272.540009,272.549988,16,4,0.139112,0.600000,0.852283,1
